In [1]:
# Benders decomposition for Fixed Charge Transportation Problem

import gurobipy as gp
from gurobipy import *
import numpy as np

In [2]:
capacity = [5,8,7]
demand = [3,6,6,5]

locations = len(capacity)
customers = len(demand)

varcost = [[4,5,2,7],[5,8,6,2],[8,9,4,3]]
fixcost = [[2,10,1,5],[7,2,7,8],[4,0,3,9]]

In [3]:
bigM = [[min(capacity[i], demand[j]) for j in range(customers)] for i in range(locations)]
bigM

[[3, 5, 5, 5], [3, 6, 6, 5], [3, 6, 6, 5]]

In [24]:
# Define Benders subproblem
        
def subproblem(model, where):
    if where == GRB.Callback.MIPSOL:
               
        v_y = model.cbGetSolution(model._y)
        print('\nCurrent y: ', v_y)
        
        LB = model.cbGetSolution(model._z)
        print('Current LB: ', LB, '\n')
        
        bsp = gp.Model("Subproblem") 
        #variables
        x = {}
        for i in range(locations):
            for j in range(customers):
                x[i,j] = bsp.addVar(obj=varcost[i][j])
        
        #demand constraint
        demand_constr = {} 
        for j in range(customers):
            demand_constr[j] = bsp.addConstr(sum(x[i,j] for i in range(locations)) >= demand[j])
        
        #capacity constraint
        cap_constr = {}
        for i in range(locations):
            cap_constr[i] = bsp.addConstr(sum(x[i,j] for j in range(customers)) <= capacity[i])
        
        #logical constraint
        log_constr = {}
        for i in range(locations):
            for j in range(customers):
                log_constr[i,j] = bsp.addConstr(x[i,j] <= v_y[i,j] * bigM[i][j])
        
        bsp.optimize()
        
        
        print('\n\n',bsp.Status,'\n\n')
        
        if bsp.Status in [3,4,5]: #infeasible or unbounded subproblem
            infbsp = gp.Model('Infeasible subproblem')
                        
            v = {}
            for j in range(customers): v[j] = infbsp.addVar()
            w = {}
            for i in range(locations):
                for j in range(customers):
                    w[i,j] = infbsp.addVar()
            u = {}
            for i in range(locations): u[i] = infbsp.addVar()
            
            infbsp.setObjective( sum(demand[j]*v[j] for j in range(customers)) - 
                              sum(capacity[i]*u[i] for i in range(locations)) - 
                              sum(bigM[i][j]*v_y[i,j]*w[i,j] for i in range(locations) for j in range(customers)))
            
            for i in range(locations):
                for j in range(customers):
                    infbsp.addConstr( v[j] - u[i] - w[i,j] >= 0);
            
            infbsp.addConstr( sum(demand[j]*v[j] for j in range(customers)) - 
                              sum(capacity[i]*u[i] for i in range(locations)) - 
                              sum(bigM[i][j]*v_y[i,j]*w[i,j] for i in range(locations) for j in range(customers)) <= LB)
            infbsp.addConstr( sum(demand[j]*v[j] for j in range(customers)) - 
                              sum(capacity[i]*u[i] for i in range(locations)) - 
                              sum(bigM[i][j]*v_y[i,j]*w[i,j] for i in range(locations) for j in range(customers)) > 0)             
            
            #infbsp.Params.DualReductions = 0
            #infbsp.modelSense = -1
            infbsp.optimize()
            print(infbsp.Status)

            #positive dual variables => use - instead of +
            model.cbLazy(sum(demand[j] * v[j].x for j in range(customers)) -    
                         sum(capacity[i] * u[i].x for i in range(locations)) -  
                         sum(bigM[i][j]*w[i,j].x*model._y[i,j] for i in range(locations) for j in range(customers)) <= 0)
            print(v)
            print(w)
            print(u)
            infbsp.dispose()
                    
                   
        else: #feasible subproblem
            print(bsp.objVal)
            
            #update ub and lb
            if bsp.objVal > LB:
                v = np.zeros(customers) #dual of demand constraints
                for j in range(customers):
                    v[j] = demand_constr[j].pi #get dual value

                w = np.zeros((locations,customers)) #dual of logical constraints
                for i in range(locations):
                    for j in range(customers):
                        w[i,j] = log_constr[i,j].pi

                u = np.zeros(locations) #dual of capacity constraints
                for i in range(locations):
                    u[i] = cap_constr[i].pi

                cm = np.zeros((locations,customers)) #coefficient of y in master problem
                for i in range(locations):
                    for j in range(customers):
                        cm[i,j] = fixcost[i][j] + bigM[i][j] * w[i,j]

                model.cbLazy(model._z >= sum(demand[j] * v[j] for j in range(customers)) +   
                         sum(capacity[i] * u[i] for i in range(locations)) + 
                         sum(cm[i,j] * model._y[i,j] for i in range(locations) for j in range(customers)))
                
                # save result of x from subproblem
                global transport
                for i in range(locations):
                    for j in range(customers):
                        transport[i,j] = x[i,j].x
                print(transport)

#            print(cm)
#            print(v)
#            print(w)
#            print(u)

        bsp.dispose()
                       

In [25]:
# Masterproblem
                     
m = gp.Model("Benders Fixed Charge Transportation Masterproblem")

y = {}
z = m.addVar(obj=1)                                                                           
for i in range(locations):
    for j in range(customers):
        y[i,j] = m.addVar(vtype=GRB.BINARY)

#for j in range(customers):
#    m.addConstr(sum(y[i,j] for i in range(locations)) >= 1)
        
m.Params.lazyConstraints = 1
m._y = y
m._z = z

transport = np.zeros((locations, customers)) # to save result of x from subproblem
m.optimize(subproblem)


print("Objective: " + str(m.objVal))

print("Solution:")
import pandas as pd
index = ['Source ' + str(x) for x in range(locations)]
columns = ['Customer ' + str(x) for x in range(customers)]

solution = pd.DataFrame(index=index, columns=columns)

for i in range(locations):
    for j in range(customers):
        solution.iloc[i,j] = transport[i,j]
        
print(solution)

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 0 rows, 13 columns and 0 nonzeros
Model fingerprint: 0xd03868d4
Variable types: 1 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]

Current y:  {(0, 0): -0.0, (0, 1): -0.0, (0, 2): -0.0, (0, 3): -0.0, (1, 0): -0.0, (1, 1): -0.0, (1, 2): -0.0, (1, 3): -0.0, (2, 0): -0.0, (2, 1): -0.0, (2, 2): -0.0, (2, 3): -0.0}
Current LB:  0.0 

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 19 rows, 12 columns and 36 nonzeros
Model fingerprint: 0xeaf8d6d6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 8e+00]
Presolve removed 12 rows and 12 columns
Presolve time: 

NotImplementedError: 

Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.04 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

User-callback calls 20, time in user-callback 0.03 sec
Objective: 0.0
Solution:
          Customer 0  Customer 1  Customer 2  Customer 3
Source 0           0           0           0           0
Source 1           0           0           0           0
Source 2           0           0           0           0


Exception ignored in: 'gurobipy.callbackstub'
Traceback (most recent call last):
  File "callback.pxi", line 180, in gurobipy.CallbackClass.callback
  File "<ipython-input-24-65ebf54cf668>", line 65, in subproblem
  File "linexpr.pxi", line 434, in gurobipy.LinExpr.__richcmp__
NotImplementedError: 


In [32]:
help(gp.Model.objBound)

AttributeError: type object 'Model' has no attribute 'objBound'